In [13]:
import pandas as pd
import sqlite3
from datetime import datetime

conn = sqlite3.connect("../data/student_roles.db")
today_year = datetime.today().year


In [15]:
emp_df = pd.read_sql_query("SELECT * FROM employee_records", conn)

def compute_employee_status(start_date):
    year = int(start_date.split("-")[0])
    return "active" if year <= today_year else "pending"

emp_df["status"] = emp_df["start_date"].apply(compute_employee_status)
emp_df.to_sql("employee_records", conn, if_exists="replace", index=False)
print("Employee statuses updated.")


Employee statuses updated.


In [19]:
stu_df = pd.read_sql_query("SELECT * FROM student_roles", conn)

def compute_student_status(role, start_date):
    year = int(start_date.split("-")[0])
    if role == "Student":
        return "active" if year <= today_year else "pending"
    return "student_worker"

stu_df["status"] = stu_df.apply(lambda row: compute_student_status(row["role"], row["start_date"]), axis=1)
stu_df.to_sql("student_roles", conn, if_exists="replace", index=False)
print("Student statuses updated.")




Student statuses updated.


In [22]:
# Generating provision_queue
active_employees = emp_df[emp_df["status"] == "active"]
active_students = stu_df[stu_df["status"] == "active"]

active_employees["type"] = "employee"
active_students.rename(columns={"student_id": "id"}, inplace=True)
active_students["type"] = "student"

active_employees = active_employees[["id", "name", "email", "department", "start_date", "type"]]
active_students = active_students[["id", "name", "email", "department", "start_date", "type"]]

provision_queue = pd.concat([active_employees, active_students])
provision_queue.to_sql("provision_queue", conn, if_exists="replace", index=False)

print("Provisioning queue created with active records only.")

conn.close()


Provisioning queue created with active records only.


/var/folders/dp/gd305wgn2g17yhrhl87p5gfr0000gn/T/ipykernel_9685/2665041321.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  active_students.rename(columns={"student_id": "id"}, inplace=True)
/var/folders/dp/gd305wgn2g17yhrhl87p5gfr0000gn/T/ipykernel_9685/2665041321.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  active_students["type"] = "student"
